# Test counters code

Try out code for disinfo countermeasures before it gets used online

In [5]:
# Try object code
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer


class Counter:
    def __init__(self, infile = 'CountersPlaybook_MASTER.xlsx'):
        
        # Load metadata from counters excelfile
        # FIXIT: Ungodly hack = please fix
        xlsx = pd.ExcelFile(infile)
        allamitts = xlsx.parse(['AMITT_objects'])
        dfa = allamitts['AMITT_objects']
        self.dftactics = dfa[3:15].copy()
        self.dfresponses = dfa[18:25].copy()
        self.dfactors = dfa[28:36].copy()
        self.dftechniques = dfa[39:100].copy()

        # Get counters data
        self.dfcounters = pd.read_excel(infile, sheet_name='Countermeasures')
        
        # Create cross-tables
        crossidtechs = self.splitcol(self.dfcounters[['ID', 'Techniques']], 
                                     'Techniques', 'Techs', '\n')
        crossidtechs = crossidtechs[crossidtechs['Techs'].notnull()]
        crossidtechs['TID'] = crossidtechs['Techs'].str.split(' ').str[0]
        crossidtechs.drop('Techs', axis=1, inplace=True)
        self.idtechnique = crossidtechs
        
        crossidres = self.splitcol(self.dfcounters[['ID', 'Resources needed']], 
                                   'Resources needed', 'Res', ',')
        crossidres = crossidres[crossidres['Res'].notnull()]
        self.idresource = crossidres

        
    def analyse_counter_text(self, col='Title'):
        # Analyse text in counter descriptions
        alltext = (' ').join(self.dfcounters[col].to_list()).lower()
        count_vect = CountVectorizer(stop_words='english')
        word_counts = count_vect.fit_transform([alltext])
        dfw = pd.DataFrame(word_counts.A, columns=count_vect.get_feature_names()).transpose()
        dfw.columns = ['count']
        dfw = dfw.sort_values(by='count', ascending=False)
        return(dfw)   

    
    def splitcol(self, df, col, newcol, divider=','):
        # Thanks https://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows?noredirect=1
        return (df.join(df[col]
                        .str.split(divider, expand=True).stack()
                        .reset_index(drop=True,level=1)
                        .rename(newcol)).drop(col, axis=1))

    
    # Print list of counters for each square of the COA matrix
    # Write HTML version of framework diagram to markdown file
    def write_coacounts_markdown(self, outfile = '../coacounts.md'):

        coacounts = pd.pivot_table(self.dfcounters[['Tactic', 'Response',
                                                    'ID']], index='Response', columns='Tactic', aggfunc=len, fill_value=0)

        html = '''# AMITT Courses of Action matrix:

    <table border="1">
    <tr>
    <td> </td>
    '''
        #Table heading = Tactic names
        for col in coacounts.columns.get_level_values(1):
            tid = self.create_tactic_file(col)
            html += '<td><a href="tactics/{0}counters.md">{1}</a></td>\n'.format(
                tid, col)
        html += '</tr><tr>\n'

        # number of counters per response type
        for response, counts in coacounts.iterrows(): 
            html += '<td>{}</td>\n'.format(response)
            for val in counts.values:
                html += '<td>{}</td>\n'.format(val)
            html += '</tr>\n<tr>\n'
        
        # Total per tactic
        html += '<td>TOTALS</td>\n'
        for val in coacounts.sum().values:
                html += '<td>{}</td>\n'.format(val)
        html += '</tr>\n</table>\n'           

        with open(outfile, 'w') as f:
            f.write(html)
            print('updated {}'.format(outfile))
        return

    def create_tactic_file(self, tname):
        if not os.path.exists('../tactics'):
            os.makedirs('../tactics')

        tid = tname[:tname.find(' ')]
        html = '''# Tactic {} counters\n\n'''.format(tname)
        
        for resp, counters in self.dfcounters[self.dfcounters['Tactic'] == tname].groupby('Response'):
            html += '\n## {}\n'.format(resp)
            
            for c in counters.iterrows():
                html += '* {}: {} (needs {})\n'.format(c[1]['ID'], c[1]['Title'],
                                                    c[1]['Resources needed'])
        datafile = '../tactics/{}_counter.md'.format(tname)
        print('Writing {}'.format(datafile))
        with open(datafile, 'w') as f:
            f.write(html)
            f.close()
        return(tid)

            
def make_object_dict(df):
    return(pd.Series(df.name.values,index=df.Id).to_dict())

counter = Counter()
counter.write_coacounts_markdown()
counter.dfcounters

Writing ../tactics/ALL_counter.md
Writing ../tactics/TA01 Strategic Planning_counter.md
Writing ../tactics/TA02 Objective Planning_counter.md
Writing ../tactics/TA03 Develop People_counter.md
Writing ../tactics/TA04 Develop Networks_counter.md
Writing ../tactics/TA05 Microtargeting_counter.md
Writing ../tactics/TA06 Develop Content_counter.md
Writing ../tactics/TA07 Channel Selection_counter.md
Writing ../tactics/TA08 Pump Priming_counter.md
Writing ../tactics/TA09 Exposure_counter.md
Writing ../tactics/TA10 Go Physical_counter.md
Writing ../tactics/TA11 Persistence_counter.md
Writing ../tactics/TA12 Measure Effectiveness_counter.md
updated ../coacounts.md


,ID,metatechnique,Title,Details,Playbook(s),Resources needed,How found,References,Incidents,Tactic,Response,Techniques
0,C00001,NaN,Better models of info spread up the layers,NaN,NaN,NaN,2019-11-workshop,NaN,NaN,ALL,ALL,NaN
1,C00002,NaN,Full spectrum analytics,NaN,NaN,data_scientist,2019-11-workshop,NaN,NaN,ALL,ALL,NaN
2,C00003,NaN,How can we safeguard against extremists using ...,NaN,NaN,NaN,2019-11-workshop,NaN,NaN,ALL,ALL,NaN
3,C00004,NaN,Managing like a chronic disease,NaN,NaN,NaN,2019-11-workshop,NaN,NaN,ALL,ALL,NaN
4,C00005,NaN,"Policy: makers, terminology, elements: a) broa...",NaN,NaN,NaN,2019-11-workshop,NaN,NaN,ALL,ALL,NaN
5,C00006,NaN,Charge for social media,No corresponding AMITT technique.,NaN,platform_admin:socialmedia,2019-11-workshop,NaN,NaN,TA01 Strategic Planning,D2 Deny,NaN
6,C00007,NaN,Create framework for BetterBusinessBureau (BBB...,No corresponding AMITT technique.,NaN,NaN,2019-11-workshop,NaN,NaN,TA01 Strategic Planning,D2 Deny,NaN
7,C00008,NaN,Create shared fact-checking database,Snopes is best-known example,NaN,factcheckers,2019-11-workshop\n2019-11-search,NaN,"I00049,I00050",TA01 Strategic Planning,D2 Deny,TA01 - Strategic Planning\nTA06 - Develop Cont...
8,C00009,resilience,Educate high profile influencers on best pract...,NaN,NaN,"influencers,educators",2019-11-workshop,NaN,NaN,TA01 Strategic Planning,D2 Deny,TA08 - Pump Priming\nT0010 - Cultivate ignoran...
9,C00010,NaN,Enhanced privacy regulation for social media,No corresponding AMITT technique.,NaN,government:policymakers,2019-11-workshop,NaN,NaN,TA01 Strategic Planning,D2 Deny,NaN


In [ ]:
#pd.pivot_table(counter.dfcounters, index=['ID', 'Tactic'], values=['Response'], aggfunc=lambda x: x['Response'].describe().values[1])
xx = pd.pivot_table(counter.idtechnique, index=['TID', 'ID'], aggfunc=len)
counter.idtechnique.groupby('TID').groups #Gives indices of counters

In [ ]:
counter.dftechniques[['Id', 'name']].merge(tc.reset_index(), left_on='Id', right_on='TID')

In [ ]:
# Create cross-tables
counter.idresource['Res'].value_counts().sort_index().reset_index()

In [ ]:
dfcounters['metatechnique'].value_counts()

In [12]:
for x,y in counter.idtechnique.groupby('TID'):
    print(x,y)

All         ID  TID
20  C00021  All
37  C00038  All
61  C00062  All
T0001         ID    TID
19  C00020  T0001
T0002         ID    TID
21  C00022  T0002
25  C00026  T0002
30  C00031  T0002
54  C00055  T0002
86  C00088  T0002
T0003         ID    TID
30  C00031  T0003
T0004         ID    TID
41  C00042  T0004
T0005          ID    TID
35   C00036  T0005
133  C00136  T0005
T0006         ID    TID
7   C00008  T0006
13  C00014  T0006
22  C00023  T0006
23  C00024  T0006
24  C00025  T0006
30  C00031  T0006
86  C00088  T0006
T0007          ID    TID
11   C00012  T0007
35   C00036  T0007
38   C00039  T0007
54   C00055  T0007
131  C00133  T0007
132  C00135  T0007
T0008         ID    TID
7   C00008  T0008
13  C00014  T0008
69  C00070  T0008
73  C00074  T0008
T0009          ID    TID
7    C00008  T0009
13   C00014  T0009
39   C00040  T0009
131  C00133  T0009
T0010          ID    TID
8    C00009  T0010
91   C00093  T0010
133  C00136  T0010
T0011          ID    TID
44   C00045  T0011
52   C00053  T001